# Quick analysis

In [1]:
## Import libraries

%load_ext autoreload
%autoreload 2

import os
import pandas as pd
import numpy as np
from datetime import date, datetime
from IPython.display import display, Markdown
from utilities import redact_small_numbers


In [2]:
# import data

df = pd.read_feather("../output/input.feather")
#df = pd.read_csv("../output/input.csv")

# for binary fields, replace zeros with null so that `count` can
binary_fields = ['elective_short_stay', 'elective_or_op']
for f in binary_fields:
    df[f] = df[f].astype('int64').replace(0,np.nan)

# treatment month
df['outpatient_covid_therapeutic_month'] = df['outpatient_covid_therapeutic_date'].astype(str).str[:-3]
df['inpatient_covid_therapeutic_month'] = df['inpatient_covid_therapeutic_date'].astype(str).str[:-3]

# latest date of inpatient records - to use as cutoff for all data
maxdate = df["any_admission_date"].max()
display(Markdown(f"Latest admission date: {maxdate}"))
display(Markdown(f"Therapeutics: {df['outpatient_covid_therapeutic_name'].unique().tolist()}"))

Latest admission date: 2022-04-25

Therapeutics: ['paxlovid', 'molnupiravir', 'sotrovimab', nan, 'remdesivir', 'casirivimab and imdevimab']

In [19]:

fields = {
'Outpatient':
    ['outpatient_covid_therapeutic_month',
     'elective_or_op',
     'elective_short_stay', 
     'daycase_admission_date',
     'elective_x892_date', 
     'elective_x292_date', 
     'hospital_attendance_date'],
'Inpatient':      
    ['inpatient_covid_therapeutic_month',
     'any_admission_date', 
     'any_admission_x892_date',
     'any_admission_x292_date']
}

for x in fields:
    f = fields[x][0]
    display(Markdown(f"## {x}"))

    # filter to treatment dates within available SUS data range
    df1 = df.copy().loc[(df[f] <= maxdate)]

    if x=="Inpatient": # don't count admissions if discharge date was after treatment date
        display(Markdown(f"Note: for inpatients, recent spells may not yet have completed so some data may be missing"))
        for c in fields[x][1:]: # for each admission type
            # compare discharge date with treatment date and remove admission date if not in window
            df1.loc[df1[f] > df1[c.replace("admission", "discharge")], c] = np.nan
            
    # create df for mabs and separate df for Antivirals
    name_field = f'{x.lower()}_covid_therapeutic_name'
    df1[name_field] = df1[name_field].astype(str).fillna("")
    df_mab = df1.loc[df1[name_field].str.contains("mab")]
    df_non_mab = df1.loc[~df1[name_field].str.contains("mab")]
    treatments = {"MABs":df_mab,
                  "Antivirals":df_non_mab}

    # Breakdown by treatment type (MABs/Avs)
    for t in treatments:
        df_t = treatments[t]
        summary1 = pd.DataFrame(df_t[fields[x]].count()).rename(columns={0:"count"})
        summary1 = redact_small_numbers(summary1, n=5, rate_column=None)
        summary1["percent"] = (100*(summary1["count"]/summary1["count"][f])).fillna(0).round(1)
        summary1.index = summary1.index.str.replace("_date","").str.replace("_month","")
        out_text = f"""Out of **{summary1["count"][0]}** patients treated in *{x}* settings with *{t}*, **{summary1["count"][1]} ({summary1["percent"][1]}%)** had *{summary1.index[1].replace("_"," ")}* in SUS"""

        summary1.to_csv((f"../output/{x}_{t}.csv"))
        display(Markdown(f"### {t}"), Markdown(out_text), summary1)

    display(Markdown("### Breakdown by region, age group and month (MABs only)"))
    # Breakdown by region, age group and month (MABs only, most inclusive hospital record field only)
    for col in ["region_covid_therapeutics","age_group", f]:
        summary2 = pd.DataFrame(df_mab.groupby(col)\
                [fields[x][0:2]].count()).rename(columns={0:"count"})
        summary2 = redact_small_numbers(summary2, n=5, rate_column=None)
        summary2["percent"] = (100*summary2[fields[x][1]]/summary2[f]).fillna(0).round(1)
        summary2.columns = summary2.columns.str.replace("_date","").str.replace("_month","")

        c_min = summary2["percent"].min()
        c_max = summary2["percent"].max()
        breakdown=col
        if col==f:
            breakdown = "treatment_month"
        out_text = f"""The percent of MABs patients treated in *{x}* settings found in SUS varied by *{breakdown}* from **{c_min}%** to **{c_max}%**"""
        summary2.to_csv((f"../output/{x}_{t}_by_{breakdown}.csv"))
        display( Markdown(out_text), summary2)

## Outpatient

### MABs

Out of **335** patients treated in *Outpatient* settings with *MABs*, **170 (50.7%)** had *elective or op* in SUS

,count,percent
outpatient_covid_therapeutic,335,100.0
elective_or_op,170,50.7
elective_short_stay,240,71.6
daycase_admission,180,53.7
elective_x892,40,11.9
elective_x292,30,9.0
hospital_attendance,40,11.9


### Antivirals

...

,inpatient_covid_therapeutic,any_admission,percent
region_covid_therapeutics,,,
East,35,10,28.6
East Midlands,35,10,28.6
London,65,30,46.2
North East,40,15,37.5
North West,35,15,42.9
South East,45,20,44.4
South West,30,15,50.0
West Midlands,35,20,57.1
Yorkshire and The Humber,30,10,33.3


The percent of MABs patients treated in *Inpatient* settings found in SUS varied by *age_group* from **33.3%** to **55.6%**

,inpatient_covid_therapeutic,any_admission,percent
age_group,,,
12-24,35,15,42.9
25-34,30,10,33.3
35-44,70,30,42.9
45-54,55,20,36.4
55-64,45,20,44.4
65-74,40,15,37.5
75+,35,15,42.9
missing,45,25,55.6


The percent of MABs patients treated in *Inpatient* settings found in SUS varied by *treatment_month* from **18.2%** to **62.5%**

,inpatient_covid_therapeutic,any_admission,percent
inpatient_covid_therapeutic_month,,,
,75,35,46.7
2021-12,40,25,62.5
2022-01,65,25,38.5
2022-02,65,25,38.5
2022-03,60,25,41.7
2022-04,55,10,18.2
